# Data cleaning: handling missing data

This notebook is an abstraction of the Kaggle's 5-Day Challenge.

The **goal** of this exercise is to clean missing entries. 

The **evaluation** of the assignment will follow:

* Design process and thinking as a data engineer.
* Validation of knowledge on the different tools and steps throughout the process.
* Storytelling and visualisation of the insights.

Exercise **workflow**:

* Import dependencies & download dataset from [here](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016/download).
* Explore missing data points and values.
* Assess the reason for the missing data points and values.
* Evaluate a method to drop the missing values.
* Evaluate a method to fill the missing values.
    
Notes:

* Write your code into the `TODO` cells.
* Feel free to choose how to present the results throughout the exercise, what libraries (e.g., seaborn, bokeh, etc.) and/or tools (e.g., PowerBI or Tableau).

## Preamble
________

In [37]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

np.random.seed(0) 

## Read data

In [38]:
org_dt = pd.read_csv("NFL_Play_ by_Play_ 2009-2018_(v5).csv")


## create a copy of original df
org_cpy = org_dt.copy()

# print sample of df
org_dt.head(5)

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0


## Data
________

**TODO**

* Download the data from [here](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016/download)
* Get some info regarding the dataframe (e.g., shape, dimensions, column names, etc.)

In [39]:
## Size, shape, dimensions of dataframe

print(" Size of data : ", org_dt.size)
print(" Shape (Row, columns) of data : ", org_dt.shape)
print(" Dimensions of data : ", org_dt.ndim)


 Size of data :  70186455
 Shape (Row, columns) of data :  (275241, 255)
 Dimensions of data :  2


In [40]:
## Column names
org_dt.columns.values

array(['play_id', 'game_id', 'home_team', 'away_team', 'posteam',
       'posteam_type', 'defteam', 'side_of_field', 'yardline_100',
       'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive',
       'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo',
       'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
       'pass_length', 'pass_location', 'air_yards', 'yards_after_catch',
       'run_location', 'run_gap', 'field_goal_result', 'kick_distance',
       'extra_point_result', 'two_point_conv_result',
       'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout',
       'timeout_team', 'td_team', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'total_home_score',
       'total_away_score', 'posteam_score', 'defteam_score',
       'score_differential', 'posteam_score_post', 'defteam_

## Exploration of missing data points and values
___

**TODO**

* How many missing values are there?
* What's the percentage of missing values?
* How many missing data points per column are there?

In [41]:
# Number of missing values for each column
missing_percent = org_dt.isnull().sum()*100 / len(org_dt)
miss_orgdt = pd.DataFrame({'missing (count)': org_dt.isnull().sum(),'missing (%)': missing_percent})
miss_orgdt.sort_values(by=['missing (%)'], ascending=False)

,missing (count),missing (%)
assist_tackle_4_team,275241,100.0
assist_tackle_4_player_name,275241,100.0
assist_tackle_4_player_id,275241,100.0
lateral_sack_player_id,275241,100.0
lateral_sack_player_name,275241,100.0
...,...,...
extra_point_prob,0,0.0
two_point_conversion_prob,0,0.0
total_home_epa,0,0.0
total_away_epa,0,0.0


In [42]:
## Total missing values

# Total values( rows * columns) 
tot_values = len(org_dt) * len(org_dt.columns)
print("Overall missing values : ",org_dt.isnull().values.sum())
print("Overall missing percent : ",round(org_dt.isnull().values.sum()*100 /tot_values),"%")


Overall missing values :  26950944
Overall missing percent :  38 %


## Assessment of missing data points and values
____
 
**TODO**

* Look at the # of missing points in all nonzero columns sorted descending. 

In [43]:
miss_nonzero_cols = org_dt.isnull().sum().sort_values(ascending=False).pipe(lambda s: s[s > 0])

print("Total columns : ", len(org_dt.columns))
print("Count of columns with missing values : ", len(miss_nonzero_cols))
print("Count of columns with no missing values : ", len(org_dt.columns) - len(miss_nonzero_cols))
print("% of columns with missing values : ",round(len(miss_nonzero_cols)/len(org_dt.columns)*100) ,"%","\n")
#miss_nonzero_cols

Total columns :  255
Count of columns with missing values :  228
Count of columns with no missing values :  27
% of columns with missing values :  89 % 



## Drop missing values
___

**TODO**

* Evaluate removing all rows with missing values
* Evaluate removing all columns with at least one missing value
* Compare the original dataframe and the filtered ones.


***------Evaluating : Removing all rows with missing values------***

In [44]:
## Total rows with >= 1 missing entry
org_transformed = org_dt
org_transformed.missrow = org_transformed.isnull().sum(axis=1)

print("Total rows in the dataset : ", len(org_dt))
print("Total rows with atleast 1 missing entry : ", len(org_transformed[org_transformed.missrow >= 1]))

## Remove column missrow
org_transformed.columns = org_transformed.columns[org_transformed.columns != "missrow"]

Total rows in the dataset :  275241
Total rows with atleast 1 missing entry :  275241


*100% of the rows have one or more missing entries so we cannot drop all the rows with missing entries*


***------Evaluating : Removing all columns with atleast one missing value------***

In [45]:
print(" Count of columns with atleast 1 missing entry : ", len(org_transformed.isnull().sum().pipe(lambda s: s[s >=1])))
print("% of columns with atleast 1 missing entry : ",round(len(org_transformed.isnull().sum().pipe(lambda s: s[s >=1]))/len(org_transformed.columns)*100) ,"%")


 Count of columns with atleast 1 missing entry :  228
% of columns with atleast 1 missing entry :  89 %


*As we can see above, 89% of the columns have atleast one missing entry.We cannot remove all of these columns*

*** Let's remove columns with missing entries more than 80% ***

In [54]:
## We can remove all the columns with 80% and beyond missing values
missing_features = missing_percent[missing_percent > 80.0].index

### Drop columns from org_transformed that have more than 80% missing values
col_exists = [col for col in missing_features if col in org_transformed.columns]
org_transformed = org_transformed.drop(col_exists, axis=1)
org_transformed.shape



(275241, 169)



*   After removing columns with more 80% of missing values. We are left with 169 columns



## Fill in missing values
_____

**TODO**

* Select a subset of the dataset
* Evaluate replacing all NaNs with 0
* Evaluate replacing all NaNs with the value of the next value in the same column
* Compare the original dataframe and the one with the filled NaNs


In [98]:
### Taking data subset - 30000 rows and 50 numerical columns

sub_dt = org_transformed.select_dtypes(include=np.number)
sub_dt = sub_dt.iloc[0:30000, 50:100]
print("Shape : ", sub_dt.shape, "\n")
sub_dt.tail(3)

Shape :  (30000, 50) 



,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,...,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,punt_inside_twenty,punt_in_endzone
29997,-0.145449,6.477596,-6.477596,NaN,NaN,0.000000,0.000000,2.270329,-2.270329,1.901762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29998,-0.145449,8.136396,-8.136396,1.518003,0.140797,1.518003,0.140797,3.788332,-3.788332,2.042559,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29999,0.222328,8.136396,-8.136396,NaN,NaN,0.000000,0.000000,3.788332,-3.788332,2.042559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
# replacing all NaNs with 0
sub_dt_zero = sub_dt.fillna(0)
#sub_dt_zero.isnull().sum()
sub_dt_zero.tail(3)

,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,...,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,punt_inside_twenty,punt_in_endzone
29997,-0.145449,6.477596,-6.477596,0.000000,0.000000,0.000000,0.000000,2.270329,-2.270329,1.901762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29998,-0.145449,8.136396,-8.136396,1.518003,0.140797,1.518003,0.140797,3.788332,-3.788332,2.042559,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29999,0.222328,8.136396,-8.136396,0.000000,0.000000,0.000000,0.000000,3.788332,-3.788332,2.042559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
#Evaluate replacing all NaNs with the value of the next value in the same column
#backward-fill propagates non-null value backwards 

sub_dt_bfill = sub_dt.fillna(method = 'bfill').fillna(0) # here we replaced all the values with bfill and any remaining value with "0"
sub_dt_bfill.tail(3)

,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,...,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,punt_inside_twenty,punt_in_endzone
29997,-0.145449,6.477596,-6.477596,1.518003,0.140797,0.000000,0.000000,2.270329,-2.270329,1.901762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29998,-0.145449,8.136396,-8.136396,1.518003,0.140797,1.518003,0.140797,3.788332,-3.788332,2.042559,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29999,0.222328,8.136396,-8.136396,0.000000,0.000000,0.000000,0.000000,3.788332,-3.788332,2.042559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***Compare original subset with subset after data imputation***

In [119]:
# creating function for the same
def cal_missing(df):
  missing_percent = df.isnull().sum()*100 / len(df)
  output = pd.DataFrame({'missing (count)': df.isnull().sum(),'missing (%)': missing_percent})
  output.sort_values(by=['missing (%)'], ascending=False)
  print(" Percentage of columns with missing entries : ",round(len(df.isnull().sum().pipe(lambda s: s[s >=1]))/len(df.columns)*100) ,"% \n")

  print(output)


In [120]:
cal_missing(sub_dt)

 Percentage of columns with missing entries :  54 % 

                         missing (count)  missing (%)
total_away_rush_epa                    0     0.000000
total_home_pass_epa                    0     0.000000
total_away_pass_epa                    0     0.000000
air_epa                            18636    62.120000
yac_epa                            18674    62.246667
comp_air_epa                        1011     3.370000
comp_yac_epa                        1027     3.423333
total_home_comp_air_epa                0     0.000000
total_away_comp_air_epa                0     0.000000
total_home_comp_yac_epa                0     0.000000
total_away_comp_yac_epa                0     0.000000
total_home_raw_air_epa                 0     0.000000
total_away_raw_air_epa                 0     0.000000
total_home_raw_yac_epa                 0     0.000000
total_away_raw_yac_epa                 0     0.000000
wp                                  1217     4.056667
def_wp                      

In [121]:
cal_missing(sub_dt_bfill)

 Percentage of columns with missing entries :  0 % 

                         missing (count)  missing (%)
total_away_rush_epa                    0          0.0
total_home_pass_epa                    0          0.0
total_away_pass_epa                    0          0.0
air_epa                                0          0.0
yac_epa                                0          0.0
comp_air_epa                           0          0.0
comp_yac_epa                           0          0.0
total_home_comp_air_epa                0          0.0
total_away_comp_air_epa                0          0.0
total_home_comp_yac_epa                0          0.0
total_away_comp_yac_epa                0          0.0
total_home_raw_air_epa                 0          0.0
total_away_raw_air_epa                 0          0.0
total_home_raw_yac_epa                 0          0.0
total_away_raw_yac_epa                 0          0.0
wp                                     0          0.0
def_wp                       